# Orthogonal RNN

Train Orthogonal RNN for MNIST classification based on [this Paper](https://arxiv.org/pdf/1901.08428.pdf)

NOTE: this example is still under development. 

## Problem Description

For each element in the input sequence, each layer computes the following function:
$$h_t=\tanh(W_{ih}x_t+b_{ih}+W_{hh}h_{t-1}+b_hh)$$

where $h_{t}$ is the hidden state at time $t$, and $h_{t-1}$ is the hidden state of the previous layer at time $t-1$ or the initial hidden state at time $o$. 

For each layer, we have the orthogonal constraint:
$$ W_{hh}^T W_{hh} = I $$

## Modules Importing
Import all necessary modules and add PyGRANSO src folder to system path. 

In [1]:
import time
import torch
import sys
## Adding PyGRANSO directories. Should be modified by user
sys.path.append('/users/5/dever120/PyGRANSO')
from pygranso.pygranso import pygranso
from pygranso.pygransoStruct import pygransoStruct 
from pygranso.private.getNvar import getNvarTorch
import torch.nn as nn
from torchvision import datasets
from torchvision.transforms import ToTensor
from pygranso.private.getObjGrad import getObjGradDL

## Data Initialization 
Specify torch device, neural network architecture, and generate data.

NOTE: please specify path for downloading data.

Use GPU for this problem. If no cuda device available, please set *device = torch.device('cpu')*

In [2]:
device = torch.device('cpu')

sequence_length = 28
input_size = 28
hidden_size = 30
num_layers = 1
num_classes = 10
batch_size = 100


double_precision = torch.double

class RNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        pass
    
    def forward(self, x):
        x = torch.reshape(x,(batch_size,sequence_length,input_size))
        # Set initial hidden and cell states 
        h0 = torch.zeros(
            self.num_layers,
            x.size(0),
            self.hidden_size
        ).to(device=device, dtype=double_precision)
        out, hidden = self.rnn(x, h0)  # out: tensor of shape (batch_size, seq_length, hidden_size)
        #Reshaping the outputs such that it can be fit into the fully connected layer
        out = self.fc(out[:, -1, :])
        return out
    
torch.manual_seed(0)

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device=device, dtype=double_precision)
model.train()

train_data = datasets.MNIST(
    root='/users/5/dever120/examples/data/mnist',
    train=True,                         
    transform=ToTensor(), 
    download=True,            
) 

loaders = {
    'train' : torch.utils.data.DataLoader(
        train_data, 
        batch_size=100, 
        shuffle=True, 
        num_workers=1
    ),
}

inputs, labels = next(iter(loaders['train']))
inputs, labels = inputs.reshape(
    -1, sequence_length, input_size
).to(device=device, dtype=double_precision), labels.to(device=device)

RuntimeError: Error downloading train-images-idx3-ubyte.gz:
Tried https://ossci-datasets.s3.amazonaws.com/mnist/, got:
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1032)>
Tried http://yann.lecun.com/exdb/mnist/, got:
HTTP Error 404: Not Found


## Function Set-Up

Encode the optimization variables, and objective and constraint functions.

Note: please strictly follow the format of comb_fn, which will be used in the PyGRANSO main algortihm.

In [ ]:
def user_fn(model,inputs,labels):
    # objective function    
    logits = model(inputs)
    criterion = nn.CrossEntropyLoss()
    f = criterion(logits, labels)

    A = list(model.parameters())[1]

    # inequality constraint
    ci = None

    # equality constraint 
    # special orthogonal group
    
    ce = pygransoStruct()

    c1_vec = (
        A.T @ A - torch.eye(hidden_size).to(
            device=device, dtype=double_precision
        )
    ).reshape(1, -1)
    
    ce.c1 = torch.linalg.vector_norm(c1_vec, 2) # l2 folding to reduce the total number of constraints

    return [f, ci, ce]

comb_fn = lambda model : user_fn(model, inputs, labels)

## User Options
Specify user-defined options for PyGRANSO 

In [ ]:
opts = pygransoStruct()
opts.torch_device = device
nvar = getNvarTorch(model.parameters())
opts.x0 = torch.nn.utils.parameters_to_vector(model.parameters()).detach().reshape(nvar, 1)
opts.opt_tol = 1e-3
opts.viol_eq_tol = 1e-4
opts.print_level = 1
opts.print_frequency = 50
opts.double_precision = True

opts.mu0 = 1

## Initial Test 
Check initial accuracy of the RNN model

In [ ]:
logits = model(inputs)
_, predicted = torch.max(logits.data, 1)
correct = (predicted == labels).sum().item()
print("Initial acc = {:.2f}%".format((100 * correct / len(inputs))))  

## Main Algorithm

In [ ]:
start = time.time()
cpu_soln = pygranso(var_spec=model, combined_fn=comb_fn, user_opts=opts)
end = time.time()
print("Total Wall Time: {}s".format(end - start))

## Train Accuracy

In [ ]:
torch.nn.utils.vector_to_parameters(cpu_soln.final.x, model.parameters())
logits = model(inputs)
_, predicted = torch.max(logits.data, 1)
correct = (predicted == labels).sum().item()
print("Final acc = {:.2f}%".format((100 * correct / len(inputs))))  
print("final feasibility = {}".format(cpu_soln.final.tve))

## CUDA Computations

In [ ]:
device = torch.device('cpu')

sequence_length = 28
input_size = 28
hidden_size = 30
num_layers = 1
num_classes = 10
batch_size = 100


double_precision = torch.double

class RNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        pass
    
    def forward(self, x):
        x = torch.reshape(x,(batch_size,sequence_length,input_size))
        # Set initial hidden and cell states 
        h0 = torch.zeros(
            self.num_layers,
            x.size(0),
            self.hidden_size
        ).to(device=device, dtype=double_precision)
        out, hidden = self.rnn(x, h0)  # out: tensor of shape (batch_size, seq_length, hidden_size)
        #Reshaping the outputs such that it can be fit into the fully connected layer
        out = self.fc(out[:, -1, :])
        return out
    
torch.manual_seed(0)

model = RNN(input_size, hidden_size, num_layers, num_classes).to(device=device, dtype=double_precision)
model.train()

train_data = datasets.MNIST(
    root='/users/5/dever120/examples/data/mnist',
    train=True,                         
    transform=ToTensor(), 
    download=True,            
) 

loaders = {
    'train' : torch.utils.data.DataLoader(
        train_data, 
        batch_size=100, 
        shuffle=True, 
        num_workers=1
    ),
}

inputs, labels = next(iter(loaders['train']))
inputs, labels = inputs.reshape(
    -1, sequence_length, input_size
).to(device=device, dtype=double_precision), labels.to(device=device)

In [ ]:
def user_fn(model,inputs,labels):
    # objective function    
    logits = model(inputs)
    criterion = nn.CrossEntropyLoss()
    f = criterion(logits, labels)

    A = list(model.parameters())[1]

    # inequality constraint
    ci = None

    # equality constraint 
    # special orthogonal group
    
    ce = pygransoStruct()

    c1_vec = (
        A.T @ A - torch.eye(hidden_size).to(
            device=device, dtype=double_precision
        )
    ).reshape(1,-1)
    
    ce.c1 = torch.linalg.vector_norm(c1_vec, 2) # l2 folding to reduce the total number of constraints

    return [f, ci, ce]

comb_fn = lambda model : user_fn(model, inputs, labels)

In [ ]:
def user_fn(model,inputs,labels):
    # objective function    
    logits = model(inputs)
    criterion = nn.CrossEntropyLoss()
    f = criterion(logits, labels)

    A = list(model.parameters())[1]

    # inequality constraint
    ci = None

    # equality constraint 
    # special orthogonal group
    
    ce = pygransoStruct()

    c1_vec = (
        A.T @ A - torch.eye(hidden_size).to(
            device=device, dtype=double_precision
        )
    ).reshape(1, -1)
    
    ce.c1 = torch.linalg.vector_norm(c1_vec, 2) # l2 folding to reduce the total number of constraints

    return [f, ci, ce]

comb_fn = lambda model : user_fn(model, inputs, labels)

In [ ]:
opts = pygransoStruct()
opts.torch_device = device
nvar = getNvarTorch(model.parameters())
opts.x0 = torch.nn.utils.parameters_to_vector(model.parameters()).detach().reshape(nvar, 1)
opts.opt_tol = 1e-3
opts.viol_eq_tol = 1e-4
opts.print_level = 1
opts.print_frequency = 50
opts.double_precision = True

opts.mu0 = 1

In [ ]:
logits = model(inputs)
_, predicted = torch.max(logits.data, 1)
correct = (predicted == labels).sum().item()
print("Initial acc = {:.2f}%".format((100 * correct / len(inputs)))) 

In [ ]:
start = time.time()
cuda_soln = pygranso(var_spec=model, combined_fn=comb_fn, user_opts=opts)
end = time.time()
print("Total Wall Time: {}s".format(end - start))

In [ ]:
torch.nn.utils.vector_to_parameters(cuda_soln.final.x, model.parameters())
logits = model(inputs)
_, predicted = torch.max(logits.data, 1)
correct = (predicted == labels).sum().item()
print("Final acc = {:.2f}%".format((100 * correct / len(inputs))))  
print("final feasibility = {}".format(cuda_soln.final.tve))

## Testing

In [ ]:
torch.testing.assert_close(
    cpu_soln.final.x,
    cuda_soln.final.x.cpu(),
    atol=1e-6,
    rtol=1e-6,
)